In [ ]:
%matplotlib ipympl
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
import os.path as path
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import ipywidgets as widgets
from ipywidgets import fixed, interact_manual, widgets
from IPython.display import display
import copy
from scipy.optimize import curve_fit
from scipy.stats import linregress
import pandas as pd
from matplotlib import rcParams
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Local module and scripts
from pyccapt.calibration.calibration import tools, share_variables
from pyccapt.calibration.calibration import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt
from pyccapt.calibration.leap_tools import ccapt_tools
from pyccapt.calibration.calibration import share_variables

In [ ]:
variables = share_variables.Variables()

In [ ]:
button = widgets.Button(description='load dataset')

@button.on_click
def open_file_on_click_r(b):
    global dataset_path
    folder_path = variables.last_directory
    script = '..//..//data_tools//run_dataset_path_qt.py'
    cmd = f"python {script} {folder_path}"
    result = subprocess.run(cmd, capture_output=True, text=True, shell=True)
    dataset_path = result.stdout.strip()
    variables.last_directory = dataset_path

button

In [ ]:
# flightPathLength = 243 # 105.35 # mm 
# # The initial value for t_0
# t0 = 33 #68.48 # ns

# dataset name
# data_115_Jul-27-2022_17-44_Powersweep3

In [ ]:
# ref_angle = 260
# ref_laser_intensity = 1.4e13
# pulse_energy = ref_laser_intensity * (10 ** ((angle - ref_angle) / (270 * 0.5)))
# # 1 μm^2 =1×10^−8 cm^2
# #Energy per pulse (J) = Power Density (W/cm^2) * Area (cm^2) * Pulse Duration (s)
# pulse_energy = pulse_energy * 6e-8 *  12e-15 * 1e12
# variables.data['pulse'] = pulse_energy

$$\textbf{You can specify the model of the TDC and dataset name down below}$$

In [ ]:
tdc, pulse_mode, flightPathLength_d, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()
display(tdc, pulse_mode)

In [ ]:
# exctract needed data from Pandas data frame as an numpy array
# create an instance of the Variables opject
variables = share_variables.Variables()
variables.pulse_mode = pulse_mode
dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path 
variables.result_data_name = 'physics_experiment_' + variables.dataset_name
variables.result_path = dataset_main_path + '/physics_experiment/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
    
# Create data farame out of hdf5 file dataset
data = data_tools.load_data(dataset_path, tdc.value, mode='processed')
# extract data from the path and create the Variable object
data_tools.extract_data(data, variables, flightPathLength_d.value, max_mc.value)

In [ ]:
print('The data will be saved on the path:', variables.result_data_path)
print('=============================')
print('The dataset name after saving is:', variables.result_data_name)
print('=============================')
print('The figures will be saved on the path:', variables.result_path)
print('=============================')
print('Total number of Ions:', len(data))
data

In [ ]:
# laser_power = np.zeros(len(data['high_voltage (V)'].to_numpy()))
# print(len(laser_power))
# laser_power[0:22100] = 50
# laser_power[22100:50000] = 60 #pJ
# laser_power[50000:100000] = 70
# laser_power[100000:150000] = 80
# laser_power[150000:200000] = 90
# laser_power[200000:250000] = 100
# laser_power[250000:300000] = 110
# laser_power[300000:350000] = 120
# laser_power[350000:400000] = 130
# laser_power[400000:450000] = 140
# laser_power[450000:500000] = 150
# laser_power[500000:550000] = 160


# data.insert(6, 'pulse (deg)', laser_power)
# del data['pulse']

In [ ]:
# the difference that we have between laser intensity
intensity = data['pulse'].to_numpy()
new_intensity = np.unique(intensity)
for i in range(len(new_intensity)):
    if i+1 < len(new_intensity):
        print(new_intensity[i+1] - new_intensity[i])

In [ ]:
intensity = data['pulse'].to_numpy()
dc_voltage = data['high_voltage (V)'].to_numpy()
mc = data['mc (Da)'].to_numpy()
def find_step_changes(array):
    step_changes = []
    prev_value = array[0]
    
    for i in range(1, len(array)):
        if array[i] > prev_value:
            step_changes.append(i)
            prev_value = array[i]

    return step_changes

def calculate_step_means(array, step_changes):
    step_means = []

    for i in range(len(step_changes) - 1):
        start_index = step_changes[i]
        end_index = step_changes[i + 1]
        step_mean = sum(array[start_index:end_index]) / (end_index - start_index)
        step_means.append(step_mean)

    # Calculate mean for the last step
    last_step_mean = sum(array[step_changes[-1]:]) / (len(array) - step_changes[-1])
    step_means.append(last_step_mean)

    return step_means

def calculate_step_values(array, step_changes):
    step_values = []

    for i in range(len(step_changes) - 1):
        start_index = step_changes[i]
        end_index = step_changes[i + 1]
        values_in_step = array[start_index:end_index]
        step_values.append(values_in_step)

    # Values for the last step
    values_in_last_step = array[step_changes[-1]:]
    step_values.append(values_in_last_step)

    return step_values

def extract_step_values(array, step_changes):
    step_values = []

    for index in step_changes:
        step_values.append(array[index])

    return step_values



steps = find_step_changes(intensity)
print(len(steps))
intensity_values = extract_step_values(intensity, steps)
# intensity_values = np.log(intensity_values)
voltage_means = calculate_step_means(dc_voltage, steps)
mc_steps = calculate_step_values(mc, steps)

# Plotting
# Create a figure and axis
fig, ax = plt.subplots()
plt.plot(voltage_means, intensity_values, marker='o', linestyle='-', color='b')

plt.xlabel('High Voltage (V)')
plt.ylabel('Laser Intensity (pJ)')
# ax.set_yscale('log')  # Set y-axis to logarithmic scale
plt.grid(True)
plt.show()

In [ ]:
ratios = []  # List to store the ratios
ratios_2 = []
for i, mc_step in enumerate(mc_steps):
    # 171fs
#     W_3pluse = mc_step[(mc_step > 61.2) & (mc_step < 61.5)]
#     W_2pluse = mc_step[(mc_step > 91.7) & (mc_step < 92.34)]
    # 12fs
    W_3pluse = mc_step[(mc_step > 61.2) & (mc_step < 61.7)]
    W_2pluse = mc_step[(mc_step > 91.7) & (mc_step < 92.3)]
    # 12Leap
    # W_3pluse = mc_step[(mc_step > 61.0) & (mc_step < 61.6)]
    # W_2pluse = mc_step[(mc_step > 91.6) & (mc_step < 92.3)]
    print('legnth:',len(W_2pluse), len(W_3pluse))
    # ratio = len(W_2pluse) / len(W_3pluse)  # Calculate mean ratio for each step
    # ratios.append(ratio)
    # ratio = len(W_3pluse) / len(W_2pluse)  # Calculate mean ratio for each step
    # ratios_2.append(ratio)
    # Define the bin size
    bin_size = 0.1
    # print(W_3pluse.max() , W_3pluse.min(), intensity_values[i])
    # print(W_2pluse.max() , W_2pluse.min(), intensity_values[i])
    num_bins = int((W_3pluse.max() - W_3pluse.min()) / bin_size)
    # print(num_bins)
    # Create histogram with specified bin size
    hist, bin_edges = np.histogram(W_3pluse, bins=num_bins)
    # Find the bin with the maximum occurrence
    max_occurrence_index = np.argmax(hist)
    # Get the value range corresponding to the bin with the maximum occurrence
    w3 = max_occurrence_height = hist[max_occurrence_index]

    num_bins = int((W_2pluse.max() - W_2pluse.min()) / bin_size)
    # Create histogram with specified bin size
    hist, bin_edges = np.histogram(W_2pluse, bins=num_bins)
    # Find the bin with the maximum occurrence
    max_occurrence_index = np.argmax(hist)
    # Get the value range corresponding to the bin with the maximum occurrence
    w2 = max_occurrence_height = hist[max_occurrence_index]
    print('peak:',w2, w3)
    ratios.append(w2/w3)
    ratios_2.append(w3/w2)
print(ratios)
print(ratios_2)
    
        



# np.save('ratio12_fs_w2_over_w3_plus_peak.npy', np.array(ratios))
# np.save('ratio12_fs__w3_over_w2_plus.npy_peak', np.array(ratios_2))
# np.save('pe_12_fs_peak.npy', intensity_values)
# np.save('ratio12_ps_w2_over_w3_plus_peak.npy', np.array(ratios))
# np.save('ratio12_ps__w3_over_w2_plus_peak.npy', np.array(ratios_2))
# np.save('pe_12_ps_peak.npy', intensity_values)
# np.save('ratio171_fs_w2_over_w3_plus_peak.npy', np.array(ratios))
# np.save('ratio171_fs__w3_over_w2_plus.npy_peak', np.array(ratios_2))
# np.save('pe_171_fs_peak.npy', intensity_values)

In [ ]:

ratio_12_fs = np.load('ratio12_fs_w2_over_w3_plus_peak.npy')
pe_12_fs = np.load('pe_12_fs_peak.npy')
# ratio_12_ps = np.load('ratio12_ps_w2_over_w3_plus_peak.npy')
# pe_12_ps = np.load('pe_12_ps_peak.npy')
ratio_171_fs = np.load('ratio171_fs_w2_over_w3_plus_peak.npy')
pe_171_fs = np.load('pe_171_fs_peak.npy')
# print(ratio_12_ps)
# print(pe_12_ps)
# Plotting
# fig, ax1 = plt.subplots(figsize=(3.5,2.8))

fig = plt.figure(figsize=(3.5,2.8))
ax1=fig.add_subplot(111, label="1")
# ax2=fig.add_subplot(111, label="2", frame_on=False)
ax1.plot(pe_12_fs/1000, ratio_12_fs, marker='o', label='12fs-pulses', color='green')

ax1.set_ylabel('W$^{2+}$ / W$^{3+}$')
ax1.set_xlabel('Pulse energy [nJ]', color='green', fontsize=8)
ax1.set_ylabel('W$^{2+}$ / W$^{3+}$', fontsize=8)
ax1.tick_params(axis='x', colors='green')  # Set Y-axis tick labels color to red
ax1.set_yscale('log')


ax2 = ax1.twiny()  # Create a secondary x-axis
ax2.plot(pe_12_ps, ratio_12_ps, marker='s', color='blue', label='ps12-pulses')  # Invisible plot to create the secondary x-axis
ax2.set_xlabel('Pulse energy [pJ]', color='blue', fontsize=8)
ax2.tick_params(axis='x', colors='blue')  # Set Y-axis tick labels color to red
ax2.tick_params(axis='y', colors='blue')  # Set Y-axis tick labels color to red

# ax2 = ax1.twiny()  # Create a secondary x-axis
# ax2.plot(pe_171_fs / 1e9, ratio_171_fs, marker='s', color='blue', label='171fs-pulses')  # Invisible plot to create the secondary x-axis
# ax2.set_xlabel('Pulse energy [nJ]', color='blue', fontsize=8)
# ax2.tick_params(axis='x', colors='blue')  # Set Y-axis tick labels color to red
# ax2.tick_params(axis='y', colors='blue')  # Set Y-axis tick labels color to red


# Combine legends from ax1 and ax2
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=8)

plt.tight_layout()
plt.savefig(variables.result_path + "temperature.png", format="png", dpi=600)
plt.savefig(variables.result_path + "temperature.svg", format="svg", dpi=600)
plt.show()

In [ ]:
# Convert lists to NumPy arrays
intensity_values_s = np.array(intensity_values)
voltage_means_s = np.array(voltage_means)

intensity_values_s = (intensity_values_s - np.min(intensity_values_s)) / (np.max(intensity_values_s) - np.min(intensity_values_s))
voltage_means_s = (np.max(voltage_means_s) - voltage_means_s) / np.max(voltage_means_s)


# voltage_means_s = np.array(voltage_reduction)
# Split the data into two ranges (0 to 19 and 20 to the end)
intensity_values_1 = intensity_values_s[:]
voltage_means_1 = voltage_means_s[:]

log_intensity_values_1 = intensity_values_1

# Define linear and quadratic functions for log-transformed data
def linear_func(x, a, b):
    return a * x + b

def square_root_func(x, a, b):
    return a * np.sqrt(x) + b

# Fit a linear function for both ranges
params_linear_log_1, _ = curve_fit(linear_func, log_intensity_values_1, voltage_means_1)
# params_linear_log_2, _ = curve_fit(linear_func_log, voltage_means_2, log_intensity_values_2)

# Fit quadratic function for both ranges
params_quadratic_log_1, _ = curve_fit(square_root_func, log_intensity_values_1, voltage_means_1)
# params_quadratic_log_2, _ = curve_fit(quadratic_func_log, voltage_means_2, log_intensity_values_2)


fig = plt.figure(figsize=(3.5,2.8))
ax=fig.add_subplot(111, label="1")
ax2=fig.add_subplot(111, label="2", frame_on=False)

# Plotting for the first range (0 to 19)
ax.scatter(log_intensity_values_1, voltage_means_1, label='Experimental data', color='b', s=6)
ax.plot(log_intensity_values_1, linear_func(log_intensity_values_1, *params_linear_log_1), label='Linear fit', color='r')
ax.plot(log_intensity_values_1, square_root_func(log_intensity_values_1, *params_quadratic_log_1), label='Square-root fit', color='g')


# plt.yscale('log')  # Set y-axis to logarithmic scale
ax.set_ylabel('Relative voltage reduction', fontsize=8)
ax.set_xlabel('Pulse energy (normalized)', fontsize=8)
ax.legend(fontsize=8)



scatter_plot = ax2.scatter(intensity_values, voltage_means, color='b')
ax2.xaxis.tick_top()
ax2.yaxis.tick_right()
ax2.xaxis.set_label_position('top') 
ax2.yaxis.set_label_position('right') 
ax2.set_ylabel('Voltage [V]')
ax2.set_xlabel('Pulse energy [pJ]')
scatter_plot.remove()

plt.grid(True)
plt.tight_layout()
rcParams['font.sans-serif'] = ['Arial']
rcParams['svg.fonttype'] = 'none'
plt.savefig(variables.result_path + "fs_PE_vs_voltage.png", format="png", dpi=600)
plt.savefig(variables.result_path + "fs_PE_vs_voltage.svg", format="svg", dpi=600)
# plt.savefig(variables.result_path + "LEAP_PE_vs_voltage.png", format="png", dpi=600)
# plt.savefig(variables.result_path + "LEAP_PE_vs_voltage.svg", format="svg", dpi=600)

plt.show()
